# Getty TGN | check on longitudes and latitudes
- 2024-05-18
- Language check on Getty ULAN data acquired via SPARQL-endpoint
- https://amandinancy16.medium.com/reverse-geocoding-with-geopy-c26cfb63f74c
- V. Martens

# Import

# Import packages

In [47]:
# handling jsons
import json
from json.decoder import JSONDecodeError

# spell check module
# import language_tool_python

# creating time stamps
import time

# importing files
import glob
import os

# progress bar
# from tqdm.notebook import tqdm
from tqdm import tqdm

# regex module
import re

# for multi-threading
from concurrent import futures
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp

# data wrangling
import pandas as pd
import numpy as np

# back up files
import pickle

# check for western spellings
from alphabet_detector import AlphabetDetector

# spacy lang detect
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

from neg_filter_aat import *

# preferences
# adjust pandas to show all cols
pd.set_option('display.max_colwidth', None)

# Import functions

In [27]:
def load_latest_file(filepath:str) -> str:
    '''
    args: string with filepath
    returns: latest file
    '''

    list_of_files = glob.glob(filepath)
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    return latest_file

# Constants

In [28]:
# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_df_errors = f'{time_stamp}_df_errors_tgn.pickle'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_df_lod_results = f'{time_stamp}_df_lod_results_tgn.pickle'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_excel_export = f'{time_stamp}_loc_check_TGN.xlsx'

print(f"{filename_df_errors}, {filename_df_lod_results}, {filename_excel_export}")

20240603-172644_df_errors_tgn.pickle, 20240603-172644_df_lod_results_tgn.pickle, 20240603-172644_loc_check_TGN.xlsx


# Import data

In [29]:
latest_picke_file = load_latest_file('data_dumps/*tgn.pickle')

# Open the file in binary mode
with open(latest_picke_file, 'rb') as file:
      
    # Call load method to deserialze
    df_tgn = pickle.load(file)
    
df_tgn.shape

data_dumps\results_df_tgn.pickle


(835649, 11)

In [30]:
df = df_tgn.head(10)

# Check cities and their names

In [33]:
import pandas as pd
# df = pd.DataFrame({'name': ['paris', 'berlin', 'london']})

tqdm.pandas()

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


# geolocator = Nominatim(user_agent="geopy.geocoders.Nominatim")
# geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)


# df['location'] = df['name'].progress_apply(geocode)




geolocator = Nominatim(user_agent="Nancy Amandi", timeout= 10)
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.1)
# df_tgn["location"] = df_tgn["coordinates"].progress_apply(rgeocode)




In [49]:
df_tgn['address'] = df_tgn.progress_apply(lambda row: geocode((row['lat.value'], row['long.value']), language='en', exactly_one=True), axis=1)

df_tgn['country'] = df_tgn['address'].astype(str).str.split(',').str[-1]


  0%|          | 0/835649 [00:00<?, ?it/s]

KeyboardInterrupt: 

3                     Yemen
4      United Arab Emirates
5      United Arab Emirates
6      United Arab Emirates
7      United Arab Emirates
8      United Arab Emirates
9                     Japan
10                   Israel
12                     Iran
13                    India
Name: address, dtype: object

In [74]:
df_tgn["coordinates"] = df_tgn['lat.value'] + ", " +df_tgn['long.value']

In [83]:
df_tgn = df_tgn.head(30)

In [23]:
tqdm.pandas()

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="Nancy Amandi", timeout= 10)
rgeocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.1)
df_tgn["location"] = df_tgn["coordinates"].progress_apply(rgeocode)
print(df.head(10))

NameError: name 'tqdm' is not defined

In [21]:

df_tgn.head(1)

NameError: name 'df_tgn' is not defined

In [20]:
# from geopy.geocoders import Nominatim

# geolocator = Nominatim(user_agent="geopy.geocoders.Nominatim")
# location = geolocator.reverse("52.509669, 13.376294")
# p

from pprint import pprint
pprint(location.raw)
print(location.address)
print(location.raw['address']['country'])
print(location.raw['address']['city'])

{'address': {'ISO3166-2-lvl4': 'DE-BE',
             'borough': 'Mitte',
             'city': 'Berlin',
             'country': 'Deutschland',
             'country_code': 'de',
             'house_number': '11',
             'postcode': '10785',
             'road': 'Potsdamer Platz',
             'suburb': 'Tiergarten'},
 'addresstype': 'building',
 'boundingbox': ['52.5085787', '52.5090046', '13.3753432', '13.3758898'],
 'class': 'building',
 'display_name': '11, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, '
                 'Deutschland',
 'importance': 9.99999999995449e-06,
 'lat': '52.5088708',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'http://osm.org/copyright',
 'lon': '13.375710625405464',
 'name': '',
 'osm_id': 216503834,
 'osm_type': 'way',
 'place_id': 128482208,
 'place_rank': 30,
 'type': 'commercial'}
11, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland
Deutschland
Berlin


In [49]:
df =df_tgn.head(1)

In [25]:
df['col_3'] = df.apply(lambda x: pandas_locater(x['lat.value'], x['long.value']), axis=1)

# df.apply(lambda x: pandas_locater(x., x.), axis=1)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=13.786&lon=46.141&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [11]:
from geopy.geocoders import Nominatim


11, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland


In [8]:
!pip install geopy

  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
